jupyter notebook

在这个示例中，我们将使用Python和PyTorch来构建和训练一个简单的CNN模型，用于对手写数字进行分类。我们将使用MNIST数据集，这是一个包含手写数字图像的常用数据集。
步骤1：准备数据
首先准备数据集。可以使用PyTorch的torchvision库来加载MNIST数据集。

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# 定义数据预处理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 下载并加载MNIST数据集
trainset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 演示环境中，减少训练集的总样本数
reduced_trainset = torch.utils.data.Subset(trainset, range(70 * 64))
trainloader = torch.utils.data.DataLoader(reduced_trainset, batch_size=64, shuffle=True)

NameError: name 'trainset' is not defined

In [ ]:
# Define data transformation for the test dataset (similar to what you did for the training dataset)
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the MNIST test dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, transform=test_transform, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

步骤2：构建CNN模型

接下来，您需要构建一个简单的CNN模型。以下是一个简化的示例：

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 13 * 13, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 32 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

步骤3：定义损失函数和优化器

在模型训练之前，您需要定义损失函数和优化器。

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

步骤4：训练模型

现在，您可以开始训练模型。在训练循环中，您将迭代遍历数据集并更新模型的权重。

In [ ]:
from tqdm.auto import tqdm

for epoch in range(4):  # 这里选择训练1个周期，您可以根据需要调整
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test data: {100 * correct / total}%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 选择一些图像进行可视化
num_samples_to_visualize = 5  # 选择要可视化的样本数量

# 将模型设置为评估模式，以确保不会进行梯度计算
net.eval()

# 创建一个新的测试数据加载器，以确保选择的图像不会影响后续测试
vis_testloader = torch.utils.data.DataLoader(testset, batch_size=num_samples_to_visualize, shuffle=True)

# 获取一批测试图像和标签
with torch.no_grad():
    data = next(iter(vis_testloader))
    inputs, labels = data
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)

# 可视化选择的图像和模型的预测结果
fig, axs = plt.subplots(1, num_samples_to_visualize, figsize=(15, 3))

for i in range(num_samples_to_visualize):
    img = inputs[i] / 2 + 0.5  # 反归一化图像数据
    img = img.numpy()
    label = labels[i].item()
    prediction = predicted[i].item()

    axs[i].imshow(np.transpose(img, (1, 2, 0)))
    axs[i].set_title(f"Label: {label}\nPredicted: {prediction}")

plt.show()